# Week 3
## Web Scrapper

import libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

create web page object handler

In [2]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url,'html5lib')

locate the table and split the rows

In [3]:
web_table = soup.table.find_all('td')

table = [i.text.rstrip() for i in web_table]
raw_postal = table[::3]
raw_borough = table[1::3]
raw_neigh = table[2::3]

create a dataframe and populate the dataframe with data in the table

In [28]:
toronto_post = pd.DataFrame(columns=['PostalCode','Borough','Neighbourhood'])

for i in range(len(raw_postal)):
    
    # if borough undefined then ignore
    if raw_borough[i] == 'Not assigned':
        continue
    
    # if more than one neighborhood exists in one postal code area, combine them
    if raw_postal[i] in toronto_post['PostalCode'].tolist():
        
        # if neighborhood undefined but borough is then neighborhood = borough
        tempNei = raw_borough[i] if raw_neigh[i] == 'Not assigned' else raw_neigh[i]
        
        # find the index of the duplicated postal
        index = toronto_post['PostalCode'].tolist().index(raw_postal[i])
        
        # combine them with comma seperated
        toronto_post.at[index, 'Neighbourhood'] += ', ' + tempNei
        continue
    
    # new entry, append to the dataframe
    toronto_post = toronto_post.append({'PostalCode':raw_postal[i], 
                                        'Borough':raw_borough[i], 
                                        'Neighbourhood':raw_neigh[i]},
                                       ignore_index=True)


__Here are some assumptions about the above data processing steps__ :
- If a postal code is duplicted, then the borough also is. (since the underlying meaning of postal code is to seperate boroughs
- The indexing of the dataframe is based on the order/ indexing of raw datas
- The raw data lists have equal length and all indexes are matched up

**take a look at what's in the dataframe**

In [29]:
toronto_post.head(10)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


**As per required, print the shape of the dataframe

In [33]:
toronto_post.shape

(103, 3)